# **Installing Packages**

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
# !pip install pyvirtualdisplay
!pip install tf-agents[reverb]
# !pip install pyglet
# !pip install gym
# !pip install keras-rl2

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [2]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
# import pyvirtualdisplay
# import reverb
import gym
import random

import tensorflow as tf
from typing import Optional, Sequence, Text, cast

import gin
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
import tensorflow_probability as tfp

from tf_agents.policies import tf_policy
from tf_agents.trajectories import policy_step
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.environments import py_environment
from tf_agents.environments import batched_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.metrics import py_metrics
from tf_agents.networks import network
from tf_agents.networks import q_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import greedy_policy, epsilon_greedy_policy
from tf_agents.policies import q_policy
from tf_agents.policies import tf_policy
from tf_agents.policies import utils as policy_utilities
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tensorflow.keras.models import Sequential
from tf_agents.typing import types
from tf_agents.utils import nest_utils
from tf_agents.distributions import masked
from tf_agents.specs import bandit_spec_utils

# from rl.agents import DQNAgent
# from rl.policy import BoltzmannQPolicy
# from rl.memory import SequentialMemory
from gym import spaces
from deck import Deck
from unoplayer import UnoPlayer
from game import Game
from scoreboard import Scoreboard

# **Creating Custom RL Environment**

In [3]:
PRINT_VERBOSE = True
PLAYER_FILENAME = "players.txt"
player_names = []
player_classes = []
GAMES = 2
players = 0
rounds_played = 0
rounds_won = 0


def load_player_data():
    r = open(PLAYER_FILENAME, 'r')
    line = r.readline().strip().split(',')
    global players
    while line[0]:
        player_names.append(line[0])
        player_classes.append(line[1] + "_unoplayer." + line[1] + "_UnoPlayer")
        line = r.readline().strip().split(',')
        players += 1

load_player_data()
s = Scoreboard(player_names.copy())

In [4]:
# class UnoEnv(gym.Env):
#   def __init__(self):
#     super(UnoEnv, self).__init__()

#     # action space = 10 number cards * 4 colors + 3 types of special cards (draw 2, reverse, skip) * 4 colors + 2 types of wild 4 cards * 4 colors 
#     self.action_space = spaces.Discrete(60)

#     # observation space = deck + player's hand, 54 possible cards each + num cards in others' deck
#     # for normalization purposes, each card is represented as 1/max_single_type
#     self.max_single_type = max(Deck.NUMBER_OF_DUP_REGULAR_CARDS, Deck.NUMBER_OF_DUP_ZERO_CARDS, Deck.NUMBER_OF_DUP_SPECIAL_CARDS, Deck.NUMBER_OF_WILD_CARDS, Deck.NUMBER_OF_WILD_D4_CARDS)
#     self.max_points = Deck.NUMBER_OF_DUP_REGULAR_CARDS * 200 + Deck.NUMBER_OF_DUP_SPECIAL_CARDS * 240 + Deck.NUMBER_OF_WILD_CARDS * 50 + Deck.NUMBER_OF_WILD_D4_CARDS * 50
#     self.num_cards = Deck.NUMBER_OF_DUP_REGULAR_CARDS * 36 + Deck.NUMBER_OF_DUP_ZERO_CARDS * 4 + Deck.NUMBER_OF_DUP_SPECIAL_CARDS * 12 + Deck.NUMBER_OF_WILD_CARDS + Deck.NUMBER_OF_WILD_D4_CARDS

#     self.g = Game(s, player_classes)
#     self.agent_index = -10000
#     for i in range(len(self.g.h)):
#       if self.g.scoreboard.get_player_list()[i] == "dqn":
#         self.agent_index = i
#         break
#     else:
#       raise ValueError("'dqn' not found in player list!")

#     self.observation_shape = 108 + self.g.scoreboard.get_num_players() - 1
#     self.observation_space = spaces.Box(low=np.zeros(self.observation_shape), high=np.ones(self.observation_shape))

#   def step(self, action):
#     # make agent make move according to implicit policy
#     game_end = False
#     agent_win = False
#     while self.g.h[self.g.curr_player].get_player_name() != "dqn":
#       game_end = self.g.make_move()
#       if game_end:
#         break

#     if not game_end:
#       game_end = self.g.make_move()
#       if game_end:
#         agent_win = True
    
#     if game_end:
#       if agent_win:
#         return self.make_observation(), self.g.round_points/self.max_points, True, {}
#       else:
#         return self.make_observation(), -1 * self.g.round_points/self.max_points, True, {}
#     else:
#       return self.make_observation(), 0, False, {}

#     # then, have all other players make turns
#     # if game doesn't end return 0
#     # if game ends and agent wins, reward is normalized amount of points earned
#     # if game ends and agent loses, reward is normalized amount of points earned by winner

#     # returns: state, reward, termination status, dict
  
#   def render(self):
#     # no graphical rendering
#     pass

#   def reset(self):
#     self.g = Game(s, player_classes)
#     self.agent_index = -10000
#     for i in range(len(self.g.h)):
#       if self.g.scoreboard.get_player_list()[i] == "dqn":
#         self.agent_index = i
#         break
#     else:
#       raise ValueError("'dqn' not found in player list!")

#     # returns: state/observation
#     return self.make_observation()
  
#   def make_observation(self):
#     out = [0] * 108

#     # check for cards in hand
#     for i in self.g.h[self.agent_index].cards:
#       if i.color == UnoPlayer.Color.NONE:
#         if i.rank == UnoPlayer.Rank.WILD:
#           out[52] += 1/self.max_single_type
#         if i.rank == UnoPlayer.Rank.WILD_D4:
#           out[53] += 1/self.max_single_type
#       else:
#         m = -10000
#         if i.rank == UnoPlayer.Rank.NUMBER:
#           m = i.number
#         if i.rank == UnoPlayer.Rank.SKIP:
#           m = 10
#         if i.rank == UnoPlayer.Rank.REVERSE:
#           m = 11
#         if i.rank == UnoPlayer.Rank.DRAW_TWO:
#           m = 12
#         if i.color == UnoPlayer.Color.RED:
#           out[m * 4] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.YELLOW:
#           out[m * 4 + 1] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.BLUE:
#           out[m * 4 + 2] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.GREEN:
#           out[m * 4 + 3] += 1/self.max_single_type

#     # check for cards in discard pile
#     for i in self.g.get_game_state().get_played_cards():
#       if i.color == UnoPlayer.Color.NONE:
#         if i.rank == UnoPlayer.Rank.WILD:
#           out[106] += 1/self.max_single_type
#         if i.rank == UnoPlayer.Rank.WILD_D4:
#           out[107] += 1/self.max_single_type
#       else:
#         m = -10000
#         if i.rank == UnoPlayer.Rank.NUMBER:
#           m = i.number
#         if i.rank == UnoPlayer.Rank.SKIP:
#           m = 10
#         if i.rank == UnoPlayer.Rank.REVERSE:
#           m = 11
#         if i.rank == UnoPlayer.Rank.DRAW_TWO:
#           m = 12
#         if i.color == UnoPlayer.Color.RED:
#           out[m * 4 + 54] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.YELLOW:
#           out[m * 4 + 55] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.BLUE:
#           out[m * 4 + 56] += 1/self.max_single_type
#         if i.color == UnoPlayer.Color.GREEN:
#           out[m * 4 + 57] += 1/self.max_single_type

#     # get amount of cards in other's hands
#     # for (i, j) in enumerate(self.g.get_game_state().get_num_cards_in_hands_of_upcoming_players()):
#     #   if i != self.agent_index:
#     #     out[108 + i] = j/self.num_cards
    
#     upcoming = self.g.get_game_state().get_num_cards_in_hands_of_upcoming_players()[:-1]
#     out += upcoming
    
#     return np.array(out)
  
#   def check_legal_action(self, action):
#     hand = self.make_observation()[:54]
#     if action < 52:
#       return hand[action] != 0
#     else:
#       if action < 56:
#         return hand[52] != 0
#       else:
#         return hand[53] != 0



In [5]:
class UnoEnv(py_environment.PyEnvironment):
  def __init__(self):
    # action space = 10 number cards * 4 colors + 3 types of special cards (draw 2, reverse, skip) * 4 colors + 2 types of wild 4 cards * 4 colors 
    self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=60, name='action')

    # observation space = deck + player's hand, 54 possible cards each + num cards in others' deck
    # for normalization purposes, each card is represented as 1/max_single_type
    self.max_single_type = max(Deck.NUMBER_OF_DUP_REGULAR_CARDS, Deck.NUMBER_OF_DUP_ZERO_CARDS, Deck.NUMBER_OF_DUP_SPECIAL_CARDS, Deck.NUMBER_OF_WILD_CARDS, Deck.NUMBER_OF_WILD_D4_CARDS)
    self.max_points = Deck.NUMBER_OF_DUP_REGULAR_CARDS * 200 + Deck.NUMBER_OF_DUP_SPECIAL_CARDS * 240 + Deck.NUMBER_OF_WILD_CARDS * 50 + Deck.NUMBER_OF_WILD_D4_CARDS * 50
    self.num_cards = Deck.NUMBER_OF_DUP_REGULAR_CARDS * 36 + Deck.NUMBER_OF_DUP_ZERO_CARDS * 4 + Deck.NUMBER_OF_DUP_SPECIAL_CARDS * 12 + Deck.NUMBER_OF_WILD_CARDS + Deck.NUMBER_OF_WILD_D4_CARDS

    self.g = Game(s, player_classes)
    self.agent_index = -10000
    for i in range(len(self.g.h)):
      if self.g.scoreboard.get_player_list()[i] == "dqn":
        self.agent_index = i
        break
    else:
      raise ValueError("'dqn' not found in player list!")

    self.observation_shape = 108 + self.g.scoreboard.get_num_players() - 1
    self._observation_spec = array_spec.BoundedArraySpec(shape=(self.observation_shape,), dtype=np.float32, minimum=0, maximum=1, name='observation')
    self._state = self.make_observation()
    self._episode_ended = False

    super().__init__()

  def action_spec(self):
    return self._action_spec
  
  def observation_spec(self):
    return self._observation_spec

  def _step(self, action):
    # print("DEBUG IN STEP - AGENT INDEX:", self.agent_index)
    # print("DEBUG - STEP TAKEN")
    if self._episode_ended:
      return self.reset()

    # print("DEBUG IN STEP - AGENT INDEX 2:", self.agent_index)
    # make agent make move according to implicit policy
    self._episode_ended = False
    agent_win = False
    # while self.g.h[self.g.curr_player].get_player_name() != "dqn":
    #   self._episode_ended = self.g.make_move()
    #   if self._episode_ended:
    #     break

    self._episode_ended = self.g.make_move(action)
    if self._episode_ended:
      agent_win = True
    
    if not agent_win:
      while self.g.h[self.g.curr_player].get_player_name() != "dqn":
        self._episode_ended = self.g.make_move()
        if self._episode_ended:
          break
    
    if self._episode_ended:
      global rounds_played
      rounds_played += 1
      if agent_win:
        global rounds_won
        rounds_won += 1
        # self._current_time_step = ts.termination(tf.convert_to_tensor(self.make_observation()), self.g.round_points/self.max_points)
        self._current_time_step = ts.termination(tf.convert_to_tensor(self.make_observation()), self.g.round_points/self.max_points)
        return self._current_time_step
      else:
        self._current_time_step = ts.termination(tf.convert_to_tensor(self.make_observation()), -1 * self.g.round_points/self.max_points)
        return self._current_time_step
    else:
      self._current_time_step = ts.transition(tf.convert_to_tensor(self.make_observation()), reward=0.0, discount=0.9)
      return self._current_time_step

    # then, have all other players make turns
    # if game doesn't end return 0
    # if game ends and agent wins, reward is normalized amount of points earned
    # if game ends and agent loses, reward is normalized amount of points earned by winner

  def _reset(self):
    # print("DEBUG - RESET")
    self.g = Game(s, player_classes)
    self.agent_index = -10000
    for i in range(len(self.g.h)):
      if self.g.scoreboard.get_player_list()[i] == "dqn":
        self.agent_index = i
        break
    else:
      raise ValueError("'dqn' not found in player list!")

    while self.g.h[self.g.curr_player].get_player_name() != "dqn":
      self._episode_ended = self.g.make_move()
      if self._episode_ended:
        break

    self._state = self.make_observation()
    # print("DEBUG - RESET STATE:", self._state)
    # print("DEBUG - RESET PLAYER 0", self.g.h[0])
    self._episode_ended = False

    # returns: state/observation
    self._current_time_step = ts.restart(tf.convert_to_tensor(self._state))
    return self._current_time_step
  
  def make_observation(self):
    out = [0] * 108

    #for i, j in enumerate(self.g.h):
      # print("DEBUG - PLAYER", i, "CARDS:", j)

    # check for cards in hand
    for i in self.g.h[self.agent_index].cards:
      if i.color == UnoPlayer.Color.NONE:
        if i.rank == UnoPlayer.Rank.WILD:
          out[52] += 1/self.max_single_type
        if i.rank == UnoPlayer.Rank.WILD_D4:
          out[53] += 1/self.max_single_type
      else:
        m = -10000
        if i.rank == UnoPlayer.Rank.NUMBER:
          m = i.number
        if i.rank == UnoPlayer.Rank.SKIP:
          m = 10
        if i.rank == UnoPlayer.Rank.REVERSE:
          m = 11
        if i.rank == UnoPlayer.Rank.DRAW_TWO:
          m = 12
        if i.color == UnoPlayer.Color.RED:
          out[m * 4] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.YELLOW:
          out[m * 4 + 1] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.BLUE:
          out[m * 4 + 2] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.GREEN:
          out[m * 4 + 3] += 1/self.max_single_type

    # print("DEBUG - DECK:", self.g.deck)
    # check for cards in discard pile
    for i in self.g.get_game_state().get_played_cards():
      if i.color == UnoPlayer.Color.NONE:
        if i.rank == UnoPlayer.Rank.WILD:
          out[106] += 1/self.max_single_type
        if i.rank == UnoPlayer.Rank.WILD_D4:
          out[107] += 1/self.max_single_type
      else:
        m = -10000
        if i.rank == UnoPlayer.Rank.NUMBER:
          m = i.number
        if i.rank == UnoPlayer.Rank.SKIP:
          m = 10
        if i.rank == UnoPlayer.Rank.REVERSE:
          m = 11
        if i.rank == UnoPlayer.Rank.DRAW_TWO:
          m = 12
        if i.color == UnoPlayer.Color.RED:
          out[m * 4 + 54] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.YELLOW:
          out[m * 4 + 55] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.BLUE:
          out[m * 4 + 56] += 1/self.max_single_type
        if i.color == UnoPlayer.Color.GREEN:
          out[m * 4 + 57] += 1/self.max_single_type

    # get amount of cards in other's hands
    # for (i, j) in enumerate(self.g.get_game_state().get_num_cards_in_hands_of_upcoming_players()):
    #   if i != self.agent_index:
    #     out[108 + i] = j/self.num_cards
    
    upcoming = self.g.get_game_state().get_num_cards_in_hands_of_upcoming_players()[:-1]
    upcoming = [i/self.num_cards for i in upcoming]
    out += upcoming
    
    return np.array(out, dtype=np.float32)
  
  def check_legal_action(self, action):
    # print("DEBUG - AGENT INDEX:", self.agent_index)
    # print("DEBUG - ACTION: ", action)
    hand = self.make_observation()[:54]
    # print("DEBUG - HAND: ", hand)

    if action <= -1 or action >= 60:
      return True

    up_card_int = -1
    up_card = self.g.up_card
    # print("DEBUG - UP CARD:", up_card)

    if up_card.color == UnoPlayer.Color.NONE:
      if up_card.rank == UnoPlayer.Rank.WILD:
        up_card_int = 52
      if up_card.rank == UnoPlayer.Rank.WILD_D4:
        up_card_int = 53
    else:
      m = -10000
      if up_card.rank == UnoPlayer.Rank.NUMBER:
        m = up_card.number
      if up_card.rank == UnoPlayer.Rank.SKIP:
        m = 10
      if up_card.rank == UnoPlayer.Rank.REVERSE:
        m = 11
      if up_card.rank == UnoPlayer.Rank.DRAW_TWO:
        m = 12
      if up_card.color == UnoPlayer.Color.RED:
        up_card_int = m * 4
      if up_card.color == UnoPlayer.Color.YELLOW:
        up_card_int = m * 4 + 1
      if up_card.color == UnoPlayer.Color.BLUE:
        up_card_int = m * 4 + 2
      if up_card.color == UnoPlayer.Color.GREEN:
        up_card_int = m * 4 + 3
    
    # print("DEBUG - UP CARD INT:", up_card_int)

    if not self.can_play_on(action, up_card_int, self.g.called_color):
      # print("DEBUG - ACTION INVALID A")
      return False

    if action < 52:
      # print("DEBUG - ACTION INVALID B")
      return hand[action] != 0
    else:
      if action < 56:
        # print("DEBUG - ACTION INVALID C")
        return hand[52] != 0
      else:
        # print("DEBUG - ACTION INVALID D")
        return hand[53] != 0
  
  def state(self):
    return self._state
  
  def can_play_on(self, index, up_card, called_color):
    result = index >= 52
    # print("DEBUG - CAN PLAY ON A", result)
    result = result or index % 4 == up_card % 4
    # print("DEBUG - CAN PLAY ON B", result)
    if called_color == UnoPlayer.Color.RED:
      result = result or index % 4 == 0
      # print("DEBUG - CAN PLAY ON C", result)
    elif called_color == UnoPlayer.Color.YELLOW:
      result = result or index % 4 == 1
      # print("DEBUG - CAN PLAY ON D", result)
    elif called_color == UnoPlayer.Color.BLUE:
      result = result or index % 4 == 2
      # print("DEBUG - CAN PLAY ON E", result)
    elif called_color == UnoPlayer.Color.GREEN:
      result = result or index % 4 == 3
      # print("DEBUG - CAN PLAY ON F", result)
    result = result or (index // 4 == up_card // 4 and index >= 40)
    # print("DEBUG - CAN PLAY ON G", result)
    result = result or (index // 4 == up_card // 4 and index < 40 and up_card < 40)
    # print("DEBUG - CAN PLAY ON H", result)
    return result
    # result = card.get_rank() == self.Rank.WILD
    # result = result or card.get_rank() == self.Rank.WILD_D4
    # result = result or card.get_color() == up_card.get_color()
    # result = result or card.get_color() == called_color
    # result = result or ((card.get_rank() == up_card.get_rank()) and (card.get_rank() != self.Rank.NUMBER))
    # result = result or (card.get_number() == up_card.get_number() and card.get_rank() == self.Rank.NUMBER and up_card.get_rank() == self.Rank.NUMBER)
    # return result
  
  # def time_step_spec(self) -> ts.TimeStep:
  #   """Describes the `TimeStep` fields returned by `step()`.
  #   Override this method to define an environment that uses non-standard values
  #   for any of the items returned by `step()`. For example, an environment with
  #   array-valued rewards.
  #   Returns:
  #     A `TimeStep` namedtuple containing (possibly nested) `ArraySpec`s defining
  #     the step_type, reward, discount, and observation structure.
  #   """
  #   print("DEBUG - ENV TIME STEP SPEC:", self.observation_spec(), self.reward_spec())
  #   return ts.time_step_spec(self.observation_spec(), self.reward_spec())

# **Setting up DQN RL Agent**

In [6]:
# credits to Nicholas Renotte

# def build_model(s=111, a=60):
#   model = Sequential()
#   model.add(tf.keras.Input(shape=(s,)))    
#   model.add(tf.keras.layers.Dense(128, activation='relu'))
#   model.add(tf.keras.layers.Dense(64, activation='relu'))
#   model.add(tf.keras.layers.Dense(a, activation='linear'))
#   return model

# model = build_model()

In [7]:
class MaxLegalGreedyPolicy (greedy_policy.GreedyPolicy):
  def __init__(self, env, policy: tf_policy.TFPolicy, name: Optional[Text] = None):
    super().__init__(policy, name)
    self.env = env

  def _distribution(self, time_step, policy_state):
    def dist_fn(dist):
      try:
        # print(tf.nest.flatten(tf.cast(dist.logits, dtype=np.float32)))
        dists = np.array(tf.cast(dist.logits, dtype=np.float32))

        actions = []
        for d in dists:
          # d = np.ravel(d)
          
          # print("DEBUG - DISTRIBUTION LOGITS:", d)
          # print("DEBUG - DISTRIBUTION LOGITS 0:", d[0])

          # print("DEBUG - DISTRIBUTION LOGITS ARGMAX:", tf.argmax(d))
          action = int(np.argmax(d[:60]))
          genv = self.env.pyenv.envs[0]
          # print("DEBUG - ENV TYPE", type(self.env))
          # print("DEBUG - PYENV TYPE", type(self.env.pyenv))
          while not genv.check_legal_action(action):
            # print("DEBUG - MODIFIFED DISTRIBUTION:", d[0])
            # print("DEBUG - DISTRIBUTION MAX:", np.max(d[0]), int(np.max(d[0])) == -1000000)
            d[action] = -1000000
            action = int(np.argmax(d[:60]))
            if int(np.max(d[:60])) == -1000000:
              action = 60
              break
          actions.append(action)


      except NotImplementedError:
        raise ValueError("Your network's distribution does not implement mode "
                         'making it incompatible with a greedy policy.'
                        ) from NotImplementedError

      # print("DEBUG - DETERMINISTIC:", tfp.distributions.Deterministic(loc=action))
      # print("DEBUG - DISTRIBUTION ACTIONS", actions)
      # print("DEBUG - DETERMINISTIC:", tfp.distributions.Deterministic(loc=tf.constant(actions, dtype=tf.int32)))
      return tfp.distributions.Deterministic(loc=tf.constant(actions, dtype=tf.int32))

    # print("DEBUG - OBSERVATION:", time_step.observation)
    # print("DEBUG - POLICY STATE:", policy_state)
    # print("DEBUG - DISTRIBUTION TIME STEP", time_step)
    distribution_step = self.wrapped_policy.distribution(
        time_step, policy_state)
    # print("DEBUG - DISTRIBUTION STEP", distribution_step)
    # print("DEBUG - DISTRIBUTION STEP ACTION PROBS: ", distribution_step.action)
    # print("DEBUG - POLICY STEP ACTION:", policy_step.PolicyStep(
    #     tf.nest.map_structure(dist_fn, distribution_step.action),
    #     distribution_step.state, distribution_step.info).action.dtype)
    # print("DEBUG - DISTRIBUTION STEP ACTION", tf.cast(tf.nest.map_structure(dist_fn, distribution_step.action), tf.int32))
    return policy_step.PolicyStep(
        tf.nest.map_structure(dist_fn, distribution_step.action),
        distribution_step.state, distribution_step.info)
  
  # def _get_initial_state(self, batch_size: int) -> types.NestedTensor:
  #   """Returns the initial state of the policy network.
  #   Args:
  #     batch_size: A constant or Tensor holding the batch size. Can be None, in
  #       which case the state will not have a batch dimension added.
  #   Returns:
  #     A nest of zero tensors matching the spec of the policy network state.
  #   """
  #   # return tensor_spec.zero_spec_nest(
  #   #     tf.TensorSpec(shape=(1,), dtype=tf.float32),
  #   #     outer_dims=None if batch_size is None else [batch_size])
  #   return tensor_spec.zero_spec_nest(
  #       self._policy_state_spec,
  #       outer_dims=None if batch_size is None else [batch_size])

In [8]:
class LegalRandomTFPolicy (random_tf_policy.RandomTFPolicy):
  def __init__(self, time_step_spec: ts.TimeStep,
               action_spec: types.NestedTensorSpec, genv, *args, **kwargs):
    self.genv = genv
    super().__init__(time_step_spec, action_spec, *args, **kwargs)

  def _action(self, time_step, policy_state, seed):
    observation_and_action_constraint_splitter = (
        self.observation_and_action_constraint_splitter)

    outer_dims = nest_utils.get_outer_shape(time_step, self._time_step_spec)
    if observation_and_action_constraint_splitter is not None:
      observation, mask = observation_and_action_constraint_splitter(
          time_step.observation)

      if self._stationary_mask is not None:
        mask = mask * self._stationary_mask

      action_spec = tensor_spec.from_spec(self.action_spec)
      action_spec = cast(tensor_spec.BoundedTensorSpec, action_spec)
      zero_logits = tf.cast(tf.zeros_like(mask), tf.float32)
      masked_categorical = masked.MaskedCategorical(zero_logits, mask)
      action_ = tf.cast(masked_categorical.sample() + action_spec.minimum,
                        action_spec.dtype)
      # print("DEBUG - EPSILON ACTION A:", action_)

      # If the action spec says each action should be shaped (1,), add another
      # dimension so the final shape is (B, 1) rather than (B,).
      if action_spec.shape.rank == 1:
        action_ = tf.expand_dims(action_, axis=-1)
        # print("DEBUG - EPSILON ACTION B:", action_)
      policy_info = tensor_spec.sample_spec_nest(
          self._info_spec, outer_dims=outer_dims)
    else:
      observation = time_step.observation
      action_spec = cast(tensor_spec.BoundedTensorSpec, self.action_spec)

      if self._accepts_per_arm_features:
        max_num_arms = action_spec.maximum - action_spec.minimum + 1
        batch_size = tf.shape(time_step.step_type)[0]
        num_actions = observation.get(
            bandit_spec_utils.NUM_ACTIONS_FEATURE_KEY,
            tf.ones(shape=(batch_size,), dtype=tf.int32) * max_num_arms)
        mask = tf.sequence_mask(num_actions, max_num_arms)
        zero_logits = tf.cast(tf.zeros_like(mask), tf.float32)
        masked_categorical = masked.MaskedCategorical(zero_logits, mask)
        action_ = tf.nest.map_structure(
            lambda t: tf.cast(masked_categorical.sample() + t.minimum, t.dtype),
            action_spec)
        # print("DEBUG - EPSILON ACTION C:", action_)
      elif self._stationary_mask is not None:
        batch_size = tf.shape(time_step.step_type)[0]
        mask = tf.tile(self._stationary_mask, [batch_size, 1])
        zero_logits = tf.cast(tf.zeros_like(mask), tf.float32)
        masked_categorical = masked.MaskedCategorical(zero_logits, mask)
        action_ = tf.cast(masked_categorical.sample() + action_spec.minimum,
                          action_spec.dtype)
        # print("DEBUG - EPSILON ACTION D:", action_)
      else:
        # action_ = tensor_spec.sample_spec_nest(
        #     self._action_spec, seed=seed, outer_dims=outer_dims)
        action = random.randint(0, self.genv.action_spec().maximum)
        if type(self.genv) == tf_py_environment.TFPyEnvironment:
          while not self.genv.pyenv.envs[0].check_legal_action(action):
            action = random.randint(0, self.genv.action_spec().maximum)
        else:
          while not self.genv.check_legal_action(action):
            action = random.randint(0, self.genv.action_spec().maximum)
        action_ = tf.constant([action])

      policy_info = tensor_spec.sample_spec_nest(
          self._info_spec, outer_dims=outer_dims)

    # Update policy info with chosen arm features.
    if self._accepts_per_arm_features:
      def _gather_fn(t):
        return tf.gather(params=t, indices=action_, batch_dims=1)
      chosen_arm_features = tf.nest.map_structure(
          _gather_fn, observation[bandit_spec_utils.PER_ARM_FEATURE_KEY])

      if policy_utilities.has_chosen_arm_features(self._info_spec):
        policy_info = policy_info._replace(
            chosen_arm_features=chosen_arm_features)

    # TODO(b/78181147): Investigate why this control dependency is required.
    def _maybe_convert_sparse_tensor(t):
      if isinstance(t, tf.SparseTensor):
        return tf.sparse.to_dense(t)
      else:
        return t
    if time_step is not None:
      with tf.control_dependencies(
          tf.nest.flatten(tf.nest.map_structure(_maybe_convert_sparse_tensor,
                                                time_step))):
        action_ = tf.nest.map_structure(tf.identity, action_)
        # print("DEBUG - EPSILON ACTION F:", action_)

    if self.emit_log_probability:
      if (self._accepts_per_arm_features
          or observation_and_action_constraint_splitter is not None
          or self._stationary_mask is not None):
        action_spec = cast(tensor_spec.BoundedTensorSpec, self.action_spec)
        log_probability = masked_categorical.log_prob(
            action_ - action_spec.minimum)
      else:
        log_probability = tf.nest.map_structure(
            lambda s: random_tf_policy._calculate_log_probability(outer_dims, s),
            self._action_spec)
      policy_info = policy_step.set_log_probability(policy_info,
                                                    log_probability)

    step = policy_step.PolicyStep(action_, policy_state, policy_info)
    return step

In [9]:
class MaxLegalEpsilonGreedyPolicy (epsilon_greedy_policy.EpsilonGreedyPolicy):
  """Returns epsilon-greedy samples of a given policy."""

  def __init__(self,
               policy: tf_policy.TFPolicy,
               genv,
               epsilon: types.FloatOrReturningFloat,
               exploration_mask: Optional[Sequence[int]] = None,
               info_fields_to_inherit_from_greedy: Sequence[Text] = (),
               name: Optional[Text] = None):
    """Builds an epsilon-greedy MixturePolicy wrapping the given policy.
    Args:
      policy: A policy implementing the tf_policy.TFPolicy interface.
      epsilon: The probability of taking the random action represented as a
        float scalar, a scalar Tensor of shape=(), or a callable that returns a
        float scalar or Tensor.
      exploration_mask: A `[0, 1]` vector describing which actions should be in
        the set of exploratory actions.
      info_fields_to_inherit_from_greedy: A list of policy info fields which
        should be copied over from the greedy action's info, even if the random
        action was taken.
      name: The name of this policy.
    Raises:
      ValueError: If epsilon is invalid.
    """
    super(MaxLegalEpsilonGreedyPolicy, self).__init__(
        policy,
        epsilon,
        exploration_mask,
        info_fields_to_inherit_from_greedy,
        name)
    try:
      observation_and_action_constraint_splitter = (
          policy.observation_and_action_constraint_splitter)
    except AttributeError:
      observation_and_action_constraint_splitter = None
    try:
      accepts_per_arm_features = policy.accepts_per_arm_features
    except AttributeError:
      accepts_per_arm_features = False
    self._greedy_policy = MaxLegalGreedyPolicy(genv, policy)
    # print("DEBUG - INIT GREEDY POLICY:", self._greedy_policy)
    self._epsilon = epsilon
    self._exploration_mask = exploration_mask
    self.info_fields_to_inherit_from_greedy = info_fields_to_inherit_from_greedy
    self._random_policy = LegalRandomTFPolicy(
        policy.time_step_spec,
        policy.action_spec,
        genv,
        emit_log_probability=policy.emit_log_probability,
        observation_and_action_constraint_splitter=(
            observation_and_action_constraint_splitter),
        accepts_per_arm_features=accepts_per_arm_features,
        stationary_mask=exploration_mask,
        info_spec=policy.info_spec)

  def _action(self, time_step, policy_state, seed):
      new_step_type = tf.data.Dataset.from_tensors(time_step.step_type).unbatch().as_numpy_iterator()
      new_reward = tf.data.Dataset.from_tensors(time_step.reward).unbatch().as_numpy_iterator()
      new_discount = tf.data.Dataset.from_tensors(time_step.discount).unbatch().as_numpy_iterator()
      new_observation = tf.data.Dataset.from_tensors(time_step.observation).unbatch().as_numpy_iterator()
      new_ts = ts.TimeStep(step_type = tf.convert_to_tensor(list(new_step_type)[0]),
                           reward = tf.convert_to_tensor(list(new_reward)[0]),
                           discount = tf.convert_to_tensor(list(new_discount)[0]),
                           observation = tf.convert_to_tensor(list(new_observation)[0]))
      time_step = new_ts
      # print("DEBUG - ACTION TIME STEP:", time_step)
      seed_stream = tfp.util.SeedStream(seed=seed, salt='epsilon_greedy')
      greedy_action = self._greedy_policy.action(time_step, policy_state)
      # print("DEBUG - GREEDY POLICY AND ACTION", self._greedy_policy, greedy_action)
      # print("DEBUG - RANDOM POLICY", self._random_policy)
      random_action = self._random_policy.action(time_step, (), seed_stream())

      # print("DEBUG - TIME STEP", time_step, self._time_step_spec)
      outer_shape = nest_utils.get_outer_shape(time_step, self._time_step_spec)
      rng = tf.random.uniform(
          outer_shape, maxval=1.0, seed=seed_stream(), name='epsilon_rng')
      cond = tf.greater_equal(rng, self._get_epsilon())

      # Selects the action/info from the random policy with probability epsilon.
      # TODO(b/133175894): tf.compat.v1.where only supports a condition which is
      # either a scalar or a vector. Use tf.compat.v2 so that it can support any
      # condition whose leading dimensions are the same as the other operands of
      # tf.where.
      # print("DEBUG - OUTER SHAPE", outer_shape, outer_shape.shape[0])
      outer_ndims = int(outer_shape.shape[0])

      if outer_ndims >= 2:
        raise ValueError(
            'Only supports batched time steps with a single batch dimension')
      # if cond:
        # print("DEBUG - GREEDY ACTION")
      # else:
        # print("DEBUG - RANDOM ACTION")
      # print("DEBUG - COND", cond)
      # print("DEBUG - G", greedy_action.action)
      # print("DEBUG - R", random_action.action)
      action = tf.nest.map_structure(lambda g, r: tf.where(cond, g, r),
                                    greedy_action.action, random_action.action)

      if greedy_action.info:
        if not random_action.info:
          raise ValueError('Incompatible info field')
        # Note that the objects in PolicyInfo may have different shapes, so we
        # need to call nest_utils.where() on each type of object.
        info = tf.nest.map_structure(lambda x, y: nest_utils.where(cond, x, y),
                                    greedy_action.info, random_action.info)
        if self._emit_log_probability:
          # At this point, info.log_probability contains the log prob of the
          # action chosen, conditioned on the policy that was chosen. We want to
          # emit the full log probability of the action, so we'll add in the log
          # probability of choosing the policy.
          random_log_prob = tf.nest.map_structure(
              lambda t: tf.math.log(tf.zeros_like(t) + self._get_epsilon()),
              info.log_probability)
          greedy_log_prob = tf.nest.map_structure(
              lambda t: tf.math.log(tf.ones_like(t) - self._get_epsilon()),
              random_log_prob)
          log_prob_of_chosen_policy = nest_utils.where(cond, greedy_log_prob,
                                                      random_log_prob)
          log_prob = tf.nest.map_structure(lambda a, b: a + b,
                                          log_prob_of_chosen_policy,
                                          info.log_probability)
          info = policy_step.set_log_probability(info, log_prob)
        # Overwrite bandit policy info type.
        if policy_utilities.has_bandit_policy_type(info, check_for_tensor=True):
          # Generate mask of the same shape as bandit_policy_type (batch_size, 1).
          # This is the opposite of `cond`, which is 1-D bool tensor (batch_size,)
          # that is true when greedy policy was used, otherwise `cond` is false.
          random_policy_mask = tf.reshape(tf.logical_not(cond),
                                          tf.shape(info.bandit_policy_type))  # pytype: disable=attribute-error
          bandit_policy_type = policy_utilities.bandit_policy_uniform_mask(
              info.bandit_policy_type, mask=random_policy_mask)  # pytype: disable=attribute-error
          info = policy_utilities.set_bandit_policy_type(
              info, bandit_policy_type)

        # TODO(b/196644931): Should not assume info is a named tuple.
        if self.info_fields_to_inherit_from_greedy:
          info = info._replace(
              **{
                  f: getattr(greedy_action.info, f)
                  for f in self.info_fields_to_inherit_from_greedy
              })
      else:
        if random_action.info:
          raise ValueError('Incompatible info field')
        info = ()

      # The state of the epsilon greedy policy is the state of the underlying
      # greedy policy (the random policy carries no state).
      # It is commonly assumed that the new policy state only depends only
      # on the previous state and "time_step", the action (be it the greedy one
      # or the random one) does not influence the new policy state.
      state = greedy_action.state

      return policy_step.PolicyStep(action, state, info)

In [10]:
class ModifiedQNetwork(q_network.QNetwork):
  

  def call(self, observation, step_type=None, network_state=(), training=False):
    """Runs the given observation through the network.

    Args:
      observation: The observation to provide to the network.
      step_type: The step type for the given observation. See `StepType` in
        time_step.py.
      network_state: A state tuple to pass to the network, mainly used by RNNs.
      training: Whether the output is being used for training.

    Returns:
      A tuple `(logits, network_state)`.
    """
    state, network_state = self._encoder(
        observation, step_type=step_type, network_state=network_state,
        training=training)
    # for i in state[0]:
    #   print("DEBUG - DTYPE", i.dtype)
    state = tf.reshape(state, (state.numpy().flatten().shape[-1]//self._encoder.layers[-1].units, state.shape[-1]))
    q_value = self._q_value_layer(state, training=training)
    return q_value, network_state

In [11]:
class MaxLegalDqnAgent(dqn_agent.DqnAgent):
  def __init__(self, 
               genv, 
               time_step_spec, 
               action_spec, 
               q_network, 
               optimizer, 
               td_errors_loss_fn, 
               train_step_counter):
    self.genv = genv
    super().__init__(time_step_spec=time_step_spec, 
                     action_spec=action_spec, 
                     q_network=q_network, 
                     optimizer=optimizer, 
                     td_errors_loss_fn=td_errors_loss_fn,
                     train_step_counter=train_step_counter)

  def _setup_policy(self, time_step_spec, action_spec,
                    boltzmann_temperature, emit_log_probability):

    policy = q_policy.QPolicy(
        time_step_spec,
        action_spec,
        q_network=self._q_network,
        emit_log_probability=emit_log_probability,
        observation_and_action_constraint_splitter=(
            self._observation_and_action_constraint_splitter))
    
    # print(policy.policy_state_spec)

    # if boltzmann_temperature is not None:
    #   collect_policy = boltzmann_policy.BoltzmannPolicy(
    #       policy, temperature=self._boltzmann_temperature)
    # else:
    #   collect_policy = MaxLegalEpsilonGreedyPolicy(
    #       policy, genv=genv, epsilon=self._epsilon_greedy)
    collect_policy = MaxLegalEpsilonGreedyPolicy(
          policy, genv=self.genv, epsilon=self._epsilon_greedy)
    # collect_policy = MaxLegalEpsilonGreedyPolicy(
    #       policy, genv=self.genv, epsilon=1)
    policy = MaxLegalGreedyPolicy(self.genv, policy)

    # Create self._target_greedy_policy in order to compute target Q-values.
    target_policy = q_policy.QPolicy(
        time_step_spec,
        action_spec,
        q_network=self._target_q_network,
        observation_and_action_constraint_splitter=(
            self._observation_and_action_constraint_splitter))
    self._target_greedy_policy = MaxLegalGreedyPolicy(self.genv, target_policy)

    return policy, collect_policy

In [12]:
train_env = tf_py_environment.TFPyEnvironment(UnoEnv())
eval_env = tf_py_environment.TFPyEnvironment(UnoEnv())
env = UnoEnv()

In [13]:
input_tensor_spec = tensor_spec.TensorSpec((108 + players,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((),
                                            tf.int32,
                                            minimum=0,
                                            maximum=59)
num_actions = 60

# class QNetwork(network.Network):

#   def __init__(self, input_tensor_spec, action_spec, num_actions=num_actions, name=None, fc_layer_params=(75, 40), activation_fn=tf.keras.activations.relu):
#     super(QNetwork, self).__init__(
#         input_tensor_spec=input_tensor_spec,
#         state_spec=(),
#         name=name)
#     self._sub_layers = [
#         tf.keras.layers.Dense(num_actions),
#     ]

#   def call(self, inputs, step_type=None, network_state=()):
#     del step_type
#     inputs = tf.cast(inputs, tf.float32)
#     for layer in self._sub_layers:
#       inputs = layer(inputs)
#     return inputs, network_state

In [14]:
hidden_layers = (128, 64)

qn = ModifiedQNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=hidden_layers)

In [15]:
train_step_counter = tf.Variable(0)

agent = MaxLegalDqnAgent(genv=train_env,
    time_step_spec=train_env.time_step_spec(),
    action_spec=train_env.action_spec(),
    q_network = qn,
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3),
    td_errors_loss_fn = common.element_wise_squared_loss,
    train_step_counter = train_step_counter)

# def build_agent(model, actions):
#     policy = MaxLegalEpsilonGreedyPolicy(q_policy.QPolicy(), UnoEnv())
#     memory = SequentialMemory(limit=50000, window_length=1)
#     dqn = DqnAgent(model=model, policy=policy, 
#                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
#     return dqn

# agent = build_agent(model, 60)
agent.initialize()

In [16]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [17]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

In [18]:
num_iterations = 50000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 3  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}
buffer_sequence_length =  2# @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}
replay_buffer_capacity = 100000 # @param {type:"integer"}

In [19]:
# table_name = 'uniform_table'
# replay_buffer_signature = tensor_spec.from_spec(
#       agent.collect_data_spec)
# replay_buffer_signature = tensor_spec.add_outer_dim(
#     replay_buffer_signature)

# table = reverb.Table(
#     table_name,
#     max_size=replay_buffer_max_length,
#     sampler=reverb.selectors.Uniform(),
#     remover=reverb.selectors.Fifo(),
#     rate_limiter=reverb.rate_limiters.MinSize(1),
#     signature=replay_buffer_signature)

# reverb_server = reverb.Server([table])

# replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
#     agent.collect_data_spec,
#     table_name=table_name,
#     sequence_length=buffer_sequence_length,
#     local_server=reverb_server)

# rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
#   replay_buffer.py_client,
#   table_name,
#   sequence_length=buffer_sequence_length)

In [20]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

random_policy = LegalRandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec(), train_env)

def collect_step(environment, policy):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  replay_buffer.add_batch(traj)

for _ in range(initial_collect_steps):
  collect_step(train_env, random_policy)

# This loop is so common in RL, that we provide standard implementations of
# these. For more details see the drivers module.

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)

iterator = iter(dataset)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [21]:
# replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
#     agent.collect_data_spec,
#     batch_size=batch_size,
#     max_length=2000)

# metric = py_metrics.AverageReturnMetric()
# observers = [metric]

In [22]:
agent.collect_data_spec

Trajectory(
{'action': BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(60, dtype=int32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(111,), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'policy_info': (),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [23]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [24]:
# #@test {"skip": true}

# py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       random_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=initial_collect_steps).run(train_env.reset())

In [25]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=buffer_sequence_length).prefetch(3)

dataset

<PrefetchDataset element_spec=(Trajectory(
{'action': TensorSpec(shape=(3, 2), dtype=tf.int32, name=None),
 'discount': TensorSpec(shape=(3, 2), dtype=tf.float32, name=None),
 'next_step_type': TensorSpec(shape=(3, 2), dtype=tf.int32, name=None),
 'observation': TensorSpec(shape=(3, 2, 111), dtype=tf.float32, name=None),
 'policy_info': (),
 'reward': TensorSpec(shape=(3, 2), dtype=tf.float32, name=None),
 'step_type': TensorSpec(shape=(3, 2), dtype=tf.int32, name=None)}), BufferInfo(ids=TensorSpec(shape=(3, 2), dtype=tf.int64, name=None), probabilities=TensorSpec(shape=(3,), dtype=tf.float32, name=None)))>

In [26]:
iterator = iter(dataset)
print(iterator)

# **Training DQN Agent**

In [27]:
#@test {"skip": true}
try:
  %%time
except:
  pass

tf.config.run_functions_eagerly(True)

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

train_env.reset()

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(train_env, agent.policy, num_eval_episodes)
returns = [avg_return]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [28]:
# # Reset the environment.
# time_step = train_env.reset()

# # Create a driver to collect experience.
# collect_driver = py_driver.PyDriver(
#     env,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       agent.collect_policy, use_tf_function=True),
#     [rb_observer],
#     max_steps=collect_steps_per_iteration)

# for _ in range(num_iterations):

#   # Collect a few steps and save to the replay buffer.
#   time_step, _ = collect_driver.run(time_step)

#   # Sample a batch of data from the buffer and update the agent's network.
#   experience, unused_info = next(iterator)
#   train_loss = agent.train(experience).loss

#   step = agent.train_step_counter.numpy()

#   if step % log_interval == 0:
#     print('step = {0}: loss = {1}'.format(step, train_loss))

#   if step % eval_interval == 0:
#     avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
#     print('step = {0}: Average Return = {1}'.format(step, avg_return))
#     returns.append(avg_return)

In [ ]:
train_env.reset()

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  # print("DEBUG - EXPERIENCE:", experience)
  train_loss = agent.train(experience)

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1:.2f}'.format(step, avg_return))
    returns.append(avg_return)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


step = 200: loss = 4.3747335439547896e-05
step = 400: loss = 9.98032046481967e-05
step = 600: loss = 0.0005578004638664424
step = 800: loss = 0.0003699044173117727
step = 1000: loss = 0.00022680610709358007
step = 1000: Average Return = -0.30
step = 1200: loss = 0.0002787472039926797
step = 1400: loss = 0.0004902112414129078
step = 1600: loss = 0.0011530338088050485


In [ ]:
class DQN_UnoPlayer(UnoPlayer):
    def play(self, hand, up_card, called_color, state):
        for i in range(len(hand)):
            if self.can_play_on(hand[i], up_card, called_color):
                return i
        return -1

    def call_color(self, hand):
        return self.Color.RED

    def can_play_on(self, card, up_card, called_color):
        result = card.get_rank() == self.Rank.WILD
        result = result or card.get_rank() == self.Rank.WILD_D4
        result = result or card.get_color() == up_card.get_color()
        result = result or card.get_color() == called_color
        result = result or ((card.get_rank() == up_card.get_rank()) and (card.get_rank() != self.Rank.NUMBER))
        result = result or (card.get_number() == up_card.get_number() and card.get_rank() == self.Rank.NUMBER and up_card.get_rank() == self.Rank.NUMBER)
        return result